# TF в Colab

In [4]:
%config IPCompleter.greedy=True  # более лучший автокомплит

In [5]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.15.0


#Так как дальше используются функции и методы из 1 версии TF, нужно запустить ноду ниже

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [7]:
tf.reset_default_graph()
a = tf.placeholder(tf.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b

In [8]:
print(c)

Tensor("matmul:0", shape=(2, 2), dtype=float32)


In [9]:
print(list(tf.get_default_graph().get_operations()[-1].inputs))
print(list(tf.get_default_graph().get_operations()[-1].outputs))

[<tf.Tensor 'Placeholder:0' shape=(2, 2) dtype=float32>, <tf.Tensor 'Variable/read:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'matmul:0' shape=(2, 2) dtype=float32>]


In [10]:
s = tf.InteractiveSession()

In [11]:
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((2, 2))})

2024-01-14 12:36:25.704773: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


array([[2., 2.],
       [2., 2.]], dtype=float32)

In [12]:
s.close()

# Оптимизация

In [13]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [14]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])

In [15]:
tf.trainable_variables()

[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

In [16]:
with tf.Session() as s:  # сессия сама закроется
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f])
        print(curr_x, curr_f)

-1.3505763 2.8500881
-1.080461 1.8240563
-0.8643688 1.1673961
-0.69149506 0.74713343
-0.5531961 0.47816542
-0.44255686 0.3060259
-0.35404548 0.19585657
-0.28323638 0.1253482
-0.22658911 0.08022285
-0.18127128 0.051342625


# TensorBoard логирование

In [17]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [19]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

In [20]:
with tf.Session() as s:
  summary_writer = tf.summary.FileWriter("logs/1", s.graph)
  s.run(tf.global_variables_initializer())
  for i in range(10):
      _, curr_summaries = s.run([step, summaries])
      summary_writer.add_summary(curr_summaries, i)
      summary_writer.flush()

Локально можно запустить  `tensorboard --logdir=./logs` в консоли.

А вот так можно запустить прямо в Google Colab:

In [ ]:
#! wget https://raw.githubusercontent.com/ZEMUSHKA/skillfactory-dl/master/setup_google_colab.py -O setup_google_colab.py -q
import setup_google_colab

# запускаем сервер tensorboard в фоне
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# открываем порт из Google Colab
setup_google_colab.expose_port_on_colab(6006)

# Обучение линейной модели

In [21]:
# модельные данные
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[0.22083778 0.78239017 0.38999298]]


In [22]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D), name="features")
target = tf.placeholder(tf.float32, shape=(None, 1), name="target")

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights


loss = tf.reduce_mean((target - predictions) ** 2, name="loss")

print(target.shape, predictions.shape, loss.shape)

(?, 1) (?, 1) ()


In [23]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [24]:
# производные это часть графа
tf.get_default_graph().get_operations()

[<tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'target' type=Placeholder>,
 <tf.Operation 'weights/Initializer/random_uniform/shape' type=Const>,
 <tf.Operation 'weights/Initializer/random_uniform/min' type=Const>,
 <tf.Operation 'weights/Initializer/random_uniform/max' type=Const>,
 <tf.Operation 'weights/Initializer/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'weights/Initializer/random_uniform/sub' type=Sub>,
 <tf.Operation 'weights/Initializer/random_uniform/mul' type=Mul>,
 <tf.Operation 'weights/Initializer/random_uniform' type=AddV2>,
 <tf.Operation 'weights' type=VariableV2>,
 <tf.Operation 'weights/Assign' type=Assign>,
 <tf.Operation 'weights/read' type=Identity>,
 <tf.Operation 'matmul' type=MatMul>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'loss' type=Mean>,
 <tf.Operation 'gradients/Shape' type=Const>,
 <tf.Operation 'gradients/gr

In [25]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss)

0.1727604
0.06485016
0.04477505
0.040616844
0.03975473
0.039575845


In [26]:
# результат обучения
curr_weights.T

array([[0.22809845, 0.7727131 , 0.40719077]], dtype=float32)

In [27]:
# правильные веса
w.T

array([[0.22083778, 0.78239017, 0.38999298]])

# Обучение в Keras

In [28]:
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
print(keras.__version__)

2.15.0


In [29]:
K.clear_session()  # очищает граф и перезапускает сессию

model = M.Sequential()
model.add(L.Dense(1, use_bias=False, input_shape=(D,)))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 3 (12.00 Byte)
Trainable params: 3 (12.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

In [31]:
model.fit(x, y, batch_size=8, epochs=10, callbacks=[keras.callbacks.TensorBoard("./logs/keras")])

Train on 1000 samples
Epoch 1/10
   8/1000 [..............................] - ETA: 11s - loss: 1.7011 - mean_squared_error: 1.7011

2024-01-14 12:39:07.996294: W tensorflow/c/c_api.cc:305] Operation '{name:'training/SGD/momentum/Assign' id:147 op device:{requested: '', assigned: ''} def:{{{node training/SGD/momentum/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/SGD/momentum, training/SGD/momentum/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-01-14 12:39:08.156515: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-01-14 12:39:08.156810: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-01-14 12:39:08.160131: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-01-14 12:39:08.161190: I external/local_tsl/tsl/profiler

1000/1000 [==============================] - 1s 504us/sample - loss: 0.5555 - mean_squared_error: 0.5555
Epoch 2/10
1000/1000 [==============================] - 0s 330us/sample - loss: 0.0588 - mean_squared_error: 0.0588
Epoch 3/10
1000/1000 [==============================] - 0s 454us/sample - loss: 0.0480 - mean_squared_error: 0.0480
Epoch 4/10
1000/1000 [==============================] - 0s 377us/sample - loss: 0.0451 - mean_squared_error: 0.0451
Epoch 5/10
1000/1000 [==============================] - 0s 280us/sample - loss: 0.0433 - mean_squared_error: 0.0433
Epoch 6/10
1000/1000 [==============================] - 0s 306us/sample - loss: 0.0421 - mean_squared_error: 0.0420
Epoch 7/10
1000/1000 [==============================] - 0s 292us/sample - loss: 0.0412 - mean_squared_error: 0.0412
Epoch 8/10
1000/1000 [==============================] - 0s 283us/sample - loss: 0.0407 - mean_squared_error: 0.0407
Epoch 9/10
1000/1000 [==============================] - 0s 273us/sample - loss: 0.0

In [34]:
s = tf.compat.v1.keras.backend.get_session()
s.run(model.weights)

[array([[0.21573575],
        [0.736718  ],
        [0.45632115]], dtype=float32)]